In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from sklearn.model_selection import train_test_split
import datetime 
pd.set_option('display.max_columns',None)
import seaborn as sns
from sklearn.preprocessing import scale, LabelEncoder, RobustScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [2]:
# load data
df = pd.read_csv('Train.csv')

In [3]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,5728806,Large private room in Nolita,4271676,Nat,Manhattan,Nolita,40.72217,-73.99481,Private room,120,7,3,2015-09-01,0.06,3,0
1,2243769,Super Bowl New York City Apartment,11460768,Brian,Manhattan,Upper West Side,40.80020,-73.96045,Entire home/apt,1500,1,0,NaN,NaN,1,0
2,35515415,COZY APT CLOSE TO LGA & JFK AND TRANSPORTATION,267193767,Jenny,Queens,East Elmhurst,40.75558,-73.89316,Entire home/apt,200,2,4,2019-07-05,4.00,1,365
3,36202006,Stream-Pressed Paradise (Laundry Room Setup),43392243,Rita,Staten Island,Concord,40.60110,-74.07830,Shared room,30,2,0,NaN,NaN,4,82
4,3780951,Charming Harlem apartment,16065171,Gina,Manhattan,Harlem,40.81022,-73.94266,Shared room,115,1,16,2019-05-26,0.27,1,365


In [8]:
df.shape

(34226, 16)

### Checking Count of Null-Values 

In [7]:
df.isnull().sum()*100/df.shape[0]

id                                 0.000000
name                               0.035061
host_id                            0.000000
host_name                          0.037983
neighbourhood_group                0.000000
neighbourhood                      0.000000
latitude                           0.000000
longitude                          0.000000
room_type                          0.000000
price                              0.000000
minimum_nights                     0.000000
number_of_reviews                  0.000000
last_review                       20.399696
reviews_per_month                 20.399696
calculated_host_listings_count     0.000000
availability_365                   0.000000
dtype: float64

### We have observed there were 20% null values in last_review and reviews_per_month columns.

In [824]:
df['last_review'].fillna('2011-03-23',inplace = True)

### We filled the null values of last_review column by taking minimum date of last_review column

In [832]:
df['days_last_review'] = (pd.DatetimeIndex(df.last_review)-pd.to_datetime('2011-03-23')).astype('str')

### we have calculated number of days since last review

In [833]:
df['days_last_review']

0        1623 days
1           0 days
2        3026 days
3           0 days
4        2986 days
           ...    
34221    2748 days
34222    3019 days
34223    2930 days
34224       0 days
34225    2072 days
Name: days_last_review, Length: 34226, dtype: object

In [834]:
df['days_last_review'] = (df['days_last_review'].apply(lambda x : x.split(" ")[0]))

In [835]:
df['days_last_review']

0        1623
1           0
2        3026
3           0
4        2986
         ... 
34221    2748
34222    3019
34223    2930
34224       0
34225    2072
Name: days_last_review, Length: 34226, dtype: object

### slicing

In [11]:
#df['days_last_review'].replace('0 da',0,inplace=True)

In [820]:
df['days_last_review'].value_counts()

0       6982
3014    1024
3022     960
3021     932
3015     613
        ... 
1161       1
1206       1
1309       1
1264       1
1816       1
Name: days_last_review, Length: 1672, dtype: int64

In [836]:
df['days_last_review'] = df['days_last_review'].astype('int64')

In [837]:
df['last_review'] = pd.to_datetime(df.last_review,format='%Y-%m-%d')

### converted last_review column in date time

In [838]:
df['last_review_year'] = df.last_review.dt.strftime('%Y')

### extracted only 'year' from last_review_year 

In [839]:
df.reviews_per_month = df.reviews_per_month.fillna(0)

### filled the null values of reviews_per_month with '0'. 

In [840]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
   
  # The math module contains a function named
  # radians which converts from degrees to radians.
  lon1 = radians(lon1)
  lon2 = radians(lon2)
  lat1 = radians(lat1)
  lat2 = radians(lat2)
   
  # Haversine formula
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

  c = 2 * asin(sqrt(a))
 
  # Radius of earth in kilometers. Use 3956 for miles
  r = 6371
   
  # calculate the result
  return(c * r)
# print(distance(lat1, lat2, lon1, lon2), "K.M")
lat2 =df.latitude.values.mean()
lon2=df.longitude.values.mean()
dist= []
for index,row in df.iterrows():
#     distance(row.latitude,lat2,row.longitude,lon2)
   dist.append(distance(row.latitude,lat2,row.longitude,lon2))

#distance =distance(df.latitude.values,lat2,  df.longitude.values, df.longitude.values.mean())
df['dist']= pd.Series(dist)


### we created the function with the help of Herversine formula which calculates the distances from lattitude and longitude

In [841]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,days_last_review,last_review_year,dist
0,5728806,Large private room in Nolita,4271676,Nat,Manhattan,Nolita,40.72217,-73.99481,Private room,120,7,3,2015-09-01,0.06,3,0,1623,2015,3.682178
1,2243769,Super Bowl New York City Apartment,11460768,Brian,Manhattan,Upper West Side,40.80020,-73.96045,Entire home/apt,1500,1,0,2011-03-23,0.00,1,0,0,2011,7.941988
2,35515415,COZY APT CLOSE TO LGA & JFK AND TRANSPORTATION,267193767,Jenny,Queens,East Elmhurst,40.75558,-73.89316,Entire home/apt,200,2,4,2019-07-05,4.00,1,365,3026,2019,5.773060
3,36202006,Stream-Pressed Paradise (Laundry Room Setup),43392243,Rita,Staten Island,Concord,40.60110,-74.07830,Shared room,30,2,0,2011-03-23,0.00,4,82,0,2011,17.770792
4,3780951,Charming Harlem apartment,16065171,Gina,Manhattan,Harlem,40.81022,-73.94266,Shared room,115,1,16,2019-05-26,0.27,1,365,2986,2019,9.059465


In [850]:
cat_col = df.select_dtypes(include = 'object')
num_col = df.select_dtypes(include = 'number')

### splitting the data into categorical and numerical types.

In [851]:
cat_col.drop(['name','host_name'],1,inplace=True)

### we observe there was no such correlation with target 

In [852]:
le = LabelEncoder()

le.fit(cat_col['neighbourhood'])
cat_col['neighbourhood']=le.transform(cat_col['neighbourhood'])
le.fit(cat_col['last_review_year'])
cat_col['last_review_year']=le.transform(cat_col['last_review_year'])

### we label enconded the 'neighbourhood' and 'last_review_year' column.

In [853]:
categorical_features_one_hot = pd.get_dummies(cat_col[['neighbourhood_group','room_type']])
categorical_features_one_hot.head()

,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,0,0,1,0,0,0,1,0
1,0,0,1,0,0,1,0,0
2,0,0,0,1,0,1,0,0
3,0,0,0,0,1,0,0,1
4,0,0,1,0,0,0,0,1


### we have done one-hot enconding on 'neighbourhood_group' and 'room_type' column.

In [854]:
cat_col= pd.concat([cat_col,categorical_features_one_hot],1)

### we merge the cat_col and categorical_features_one_hot into dataframe and dropped the original columns from the data. 

In [855]:
cat_col.drop(['neighbourhood_group','room_type'],1,inplace=True)

In [856]:
num_col.drop(['id','host_id'],1,inplace=True)

### id and host_id were also not having the correlation with target so we decided to drop them.

In [857]:
df_1= pd.concat([cat_col,num_col],1)

### we merge the numerical and categorical data into  data frame.   

In [858]:
df_1

,neighbourhood,last_review_year,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,days_last_review,dist
0,143,4,0,0,1,0,0,0,1,0,40.72217,-73.99481,120,7,3,0.06,3,0,1623,3.682178
1,200,0,0,0,1,0,0,1,0,0,40.80020,-73.96045,1500,1,0,0.00,1,0,0,7.941988
2,59,8,0,0,0,1,0,1,0,0,40.75558,-73.89316,200,2,4,4.00,1,365,3026,5.773060
3,46,0,0,0,0,0,1,0,0,1,40.60110,-74.07830,30,2,0,0.00,4,82,0,17.770792
4,93,8,0,0,1,0,0,0,0,1,40.81022,-73.94266,115,1,16,0.27,1,365,2986,9.059465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34221,164,7,0,0,0,1,0,1,0,0,40.70602,-73.90485,80,5,2,0.21,1,0,2748,4.732876
34222,200,8,0,0,1,0,0,0,1,0,40.77697,-73.97698,137,1,163,3.55,1,32,3019,5.725823
34223,102,8,0,0,1,0,0,1,0,0,40.86912,-73.92183,195,2,6,0.53,1,0,2930,15.780767
34224,28,0,0,1,0,0,0,0,1,0,40.69709,-73.91435,80,3,0,0.00,1,0,0,4.769012


In [859]:
X = df_1.drop('price',1)
y = df_1['price']

### splitting dependent and independent features(columns) in 'X' and 'y'.

In [860]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=33)

### using train_test_split we splitted the data into train and test data.

# Model Evaluation

### Applying vanila model 'Linear Regression'

In [861]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
#y_pred = lr.predict(X_val)
y_pred = lr.predict(X_test)

#y_pred = np.exp(y_pred)-1
#y_pred1 = np.exp(y_pred1)-1
#print('Val RMSE:',np.sqrt(mean_squared_error(y_val,y_pred)))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test,y_pred)))

Test RMSE: 226.47744755765072


### Applying Random Forest with Randomized Search CV

In [865]:
from sklearn.ensemble import GradientBoostingRegressor ,RandomForestRegressor
rf_model = RandomForestRegressor(random_state=33)
param_grid = dict(n_estimators=range(10,100,10),max_depth=range(3,20),min_samples_split=range(50,500,20),min_samples_leaf=range(25,75,10),max_leaf_nodes=range(8,32,2))
grid_rf = RandomizedSearchCV(rf_model,param_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=5,random_state=33)
grid_rf.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [866]:
#rf_pred = grid_rf.best_estimator_.predict(X_val)
rf_pred1 = grid_rf.best_estimator_.predict(X_test)

#rf_pred = np.exp(rf_pred)-1
#rf_pred1 = np.exp(rf_pred1)-1
print(grid_rf.best_params_)

#print('Test RMSLE:',np.sqrt(mean_squared_log_error(y_test, rf_pred)))
#print('Val RMSE:',np.sqrt(mean_squared_error(y_val, rf_pred)))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, rf_pred1)))
#print("R2_score: ",r2_score(y_test,rf_pred))

{'n_estimators': 70, 'min_samples_split': 70, 'min_samples_leaf': 25, 'max_leaf_nodes': 28, 'max_depth': 18}
Test RMSE: 219.66977980124636


### Applying Decision Tree with Randomized Search CV

In [867]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
dt_reg = DecisionTreeRegressor(random_state=42)
param_grid = dict(max_depth=range(5,20), min_samples_split=range(50,200,10), min_samples_leaf=range(25,100,10), max_leaf_nodes=range(8,32,2), min_impurity_decrease=(0.3,1.0,0.1))
grid_dt = RandomizedSearchCV(dt_reg, param_grid, scoring='neg_mean_squared_error', n_jobs=4, cv=5, random_state=33)
grid_dt.fit(X_train,y_train)
#dt_pred = grid_dt.best_estimator_.predict(X_val)
dt_pred1 = grid_dt.best_estimator_.predict(X_test)
#dt_pred = np.exp(dt_pred)-1
#dt_pred1 = np.exp(dt_pred1)-1
print(grid_dt.best_params_)

#print('Test RMSLE:',np.sqrt(mean_squared_log_error(y_test, dt_pred)))
#print('Val RMSE:',np.sqrt(mean_squared_error(y_val, dt_pred)))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, dt_pred1)))
#print("R2_score: ",r2_score(y_test,dt_pred))


{'min_samples_split': 170, 'min_samples_leaf': 45, 'min_impurity_decrease': 0.1, 'max_leaf_nodes': 14, 'max_depth': 11}
Test RMSE: 223.123731455727


### Applying Voting Regressor

In [29]:
# Import libraries
from sklearn.ensemble import VotingRegressor

# Various models
clf_1 = LinearRegression()
clf_2 = XGBRegressor(random_state=4)
clf_3 = RandomForestRegressor(random_state=4)

model_list = [('lr',clf_1),('DT',clf_2),('RF',clf_3)]

# Code starts here
# Initialize voting classifier
voting_clf_hard = VotingRegressor(estimators=model_list)
# Fit the model on training data
voting_clf_hard.fit(X_train,y_train)

# predict on test
y_pred_vot = voting_clf_hard.predict(X_test)
#y_pred_vot =np.exp(y_pred_vot)-1
# accuracy
hard_voting_score = voting_clf_hard.score(X_test,y_test)

print(np.sqrt(mean_squared_error(y_test,y_pred_vot)))

[07:21:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
215.04930883071177


In [769]:
xgb_pred1

array([103.76814 ,  75.54399 , 271.84134 , ...,  67.751526,  88.27993 ,
        64.76578 ], dtype=float32)

### Applying Gradient Boosting Regressor

In [327]:
from xgboost import XGBRegressor 
grid_xgb = XGBRegressor(objective='reg:squarederror',n_jobs=-1,subsample = 0.7, scale_pos_weight= 5, reg_lambda= 4, n_estimators = 110, min_child_weight= 13, max_depth = 12, learning_rate= 0.1, colsample_bytree = 0.9, colsample_bynode= 0.9)
grid_xgb.fit(X_train,y_train)
#xgb_pred = grid_xgb.best_estimator_.predict(X_val)
xgb_pred1 = grid_xgb.predict(X_test)

#xgb_pred = np.exp(xgb_pred)-1
#xgb_pred1 = np.exp(xgb_pred1)-1
#print(grid_xgb.best_params_)
#print(grid_xgb.best_estimator_)

#print('Test RMSLE:',np.sqrt(mean_squared_log_error(y_test, xgb_pred)))
#print('Val RMSE:',np.sqrt(mean_squared_error(y_val, xgb_pred)))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, xgb_pred1)))
#3print("R2_score: ",r2_score(y_test,xgb_pred))

Test RMSE: 214.20332257662415


### Final Model Evaluation Selection:-
### We observed that our XGB Model has performed good as compared to other models with the least RMSE of 214.20

# Test Data Evaluation


### Will perform same modifications on test data or validation data which earlier we done on training data.

In [14]:
df_t = pd.read_csv('Test.csv')

In [895]:
df_t.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1525602,Perfect Temporary Brooklyn Home,1200603,Andrea,Brooklyn,Crown Heights,40.66751,-73.95867,Entire home/apt,2,121,2019-05-31,1.70,1,39
1,30430185,BIG APPLE - COMFY KING ROOM,224414117,Gabriel,Manhattan,Hell's Kitchen,40.75655,-73.99690,Private room,1,18,2019-05-05,2.49,30,364
2,21354525,Cozy Brooklyn Heights Getaway w/ Manhattan Access,11743513,Henry,Brooklyn,Brooklyn Heights,40.69252,-73.99121,Private room,1,87,2019-06-16,4.29,1,108
3,35995074,Amazing*Quiet*Sunny*Bedroom*PrivateLivingRm*Ch...,4128829,Sara,Queens,Ditmars Steinway,40.77292,-73.90101,Private room,9,0,NaN,NaN,2,365
4,34392081,Hello! This is a very cozy space in Williamsburg.,259630588,Alina,Brooklyn,Williamsburg,40.71863,-73.94980,Private room,1,28,2019-06-26,14.00,2,20


In [896]:
df_ID = df_t['id']

In [897]:
df_t['last_review'].fillna('2011-03-23',inplace = True)

In [898]:
df_t['days_last_review'] = (pd.DatetimeIndex(df_t.last_review)-pd.to_datetime('2011-03-23')).astype('str')

In [900]:
df_t['days_last_review'] = (df_t['days_last_review'].apply(lambda x : x.split(" ")[0]))

In [901]:
df_t['days_last_review'] = df_t['days_last_review'].astype('int64')

In [902]:
df_t['last_review'] = pd.to_datetime(df_t.last_review,format='%Y-%m-%d')

In [903]:
df_t['last_review_year'] = df_t.last_review.dt.strftime('%Y')

In [904]:
df_t.reviews_per_month = df_t.reviews_per_month.fillna(0)

In [905]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
   
  # The math module contains a function named
  # radians which converts from degrees to radians.
  lon1 = radians(lon1)
  lon2 = radians(lon2)
  lat1 = radians(lat1)
  lat2 = radians(lat2)
   
  # Haversine formula
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

  c = 2 * asin(sqrt(a))
 
  # Radius of earth in kilometers. Use 3956 for miles
  r = 6371
   
  # calculate the result
  return(c * r)
# print(distance(lat1, lat2, lon1, lon2), "K.M")
lat2 =df.latitude.values.mean()
lon2=df.longitude.values.mean()
dist= []
for index,row in df.iterrows():
#     distance(row.latitude,lat2,row.longitude,lon2)
   dist.append(distance(row.latitude,lat2,row.longitude,lon2))

#distance =distance(df.latitude.values,lat2,  df.longitude.values, df.longitude.values.mean())
df_t['dist']= pd.Series(dist)


In [906]:
cat_col = df_t.select_dtypes(include = 'object')
num_col = df_t.select_dtypes(include = 'number')

In [907]:
cat_col.drop(['name','host_name'],1,inplace=True)

In [908]:
le = LabelEncoder()

le.fit(cat_col['neighbourhood'])
cat_col['neighbourhood']=le.transform(cat_col['neighbourhood'])
le.fit(cat_col['last_review_year'])
cat_col['last_review_year']=le.transform(cat_col['last_review_year'])

In [909]:
categorical_features_one_hot = pd.get_dummies(cat_col[['neighbourhood_group','room_type']])
categorical_features_one_hot.head()

,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,0,1,0,0,0,1,0,0
1,0,0,1,0,0,0,1,0
2,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0


In [910]:
cat_col= pd.concat([cat_col,categorical_features_one_hot],1)

In [911]:
cat_col.drop(['neighbourhood_group','room_type'],1,inplace=True)

In [912]:
num_col.drop(['id','host_id'],1,inplace=True)

In [913]:
df_1= pd.concat([cat_col,num_col],1)

In [914]:
df_1

,neighbourhood,last_review_year,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,days_last_review,dist
0,49,8,0,1,0,0,0,1,0,0,40.66751,-73.95867,2,121,1.70,1,39,2991,3.682178
1,93,8,0,0,1,0,0,0,1,0,40.75655,-73.99690,1,18,2.49,30,364,2965,7.941988
2,24,8,0,1,0,0,0,0,1,0,40.69252,-73.99121,1,87,4.29,1,108,3007,5.773060
3,52,0,0,0,0,1,0,0,1,0,40.77292,-73.90101,9,0,0.00,2,365,0,17.770792
4,200,8,0,1,0,0,0,0,1,0,40.71863,-73.94980,1,28,14.00,2,20,3017,9.059465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14664,165,8,0,1,0,0,0,1,0,0,40.59446,-73.94289,2,1,0.59,1,36,2980,8.467656
14665,109,8,0,0,1,0,0,0,0,1,40.74319,-73.98074,2,1,0.16,1,365,2841,9.004276
14666,92,8,0,0,1,0,0,0,1,0,40.82519,-73.94530,1,93,1.34,3,358,3023,4.673950
14667,139,8,0,0,1,0,0,1,0,0,40.72238,-73.99466,1,15,1.12,8,34,2987,8.747993


In [915]:
y_pred_test = grid_xgb.predict(df_1) 

In [916]:
y_pred_test

array([149.68445, 198.8911 ,  90.25699, ..., 101.78211, 237.0191 ,
       135.70839], dtype=float32)

In [917]:
df_t = pd.concat([df_ID,df_1],1)

In [918]:
submissions = pd.DataFrame(y_pred_test,index = df_t['id'],columns = ['price'])

In [919]:
submissions.head()

,price
id,
1525602,149.684448
30430185,198.891098
21354525,90.256989
35995074,101.806229
34392081,70.790749


In [920]:
submissions.to_csv('submission_stayze_325.csv', index=True)

### Creating the final submission file in the form of csv which holds the predicted results.